# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv

In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
price_data = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(price_data, "**/*.parquet"), recursive = True)
print(f"Found {len(parquet_files)} parquet files.")
for file_path in parquet_files:
    print(file_path)

Found 3261 parquet files.
../../05_src/data/prices\ACN\ACN_2001\part.0.parquet
../../05_src/data/prices\ACN\ACN_2001\part.1.parquet
../../05_src/data/prices\ACN\ACN_2002\part.0.parquet
../../05_src/data/prices\ACN\ACN_2002\part.1.parquet
../../05_src/data/prices\ACN\ACN_2003\part.0.parquet
../../05_src/data/prices\ACN\ACN_2003\part.1.parquet
../../05_src/data/prices\ACN\ACN_2004\part.0.parquet
../../05_src/data/prices\ACN\ACN_2004\part.1.parquet
../../05_src/data/prices\ACN\ACN_2005\part.0.parquet
../../05_src/data/prices\ACN\ACN_2005\part.1.parquet
../../05_src/data/prices\ACN\ACN_2006\part.0.parquet
../../05_src/data/prices\ACN\ACN_2006\part.1.parquet
../../05_src/data/prices\ACN\ACN_2007\part.0.parquet
../../05_src/data/prices\ACN\ACN_2007\part.1.parquet
../../05_src/data/prices\ACN\ACN_2008\part.0.parquet
../../05_src/data/prices\ACN\ACN_2008\part.1.parquet
../../05_src/data/prices\ACN\ACN_2009\part.0.parquet
../../05_src/data/prices\ACN\ACN_2009\part.1.parquet
../../05_src/data/pr

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
dd_feat = dd.read_parquet(parquet_files).set_index("ticker")
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ACY,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


In [10]:
import pandas as pd
# Add lags for Close and Adj_Close
dd_feat = dd.read_parquet(parquet_files).set_index("ticker")
dd_engg = (
    dd_feat
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1))
                       .assign(Adj_Close_lag_1 = x['Adj Close'].shift(1))
                       .assign(returns=lambda x: x['Close']/x['Close_lag_1'] - 1)
                       .assign(hi_lo_range = lambda x: x['High'] - x['Low']),
                       meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8',
                    'returns': 'f8',
                    'hi_lo_range': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))
dd_feat = dd_engg
print(dd_feat.head())

             Date   Open   High    Low  Close  Adj Close      Volume   source  \
ticker                                                                          
ACN    2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
ACN    2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
ACN    2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
ACN    2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
ACN    2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   

        Year  Close_lag_1  Adj_Close_lag_1   returns  hi_lo_range  
ticker                                                             
ACN     2001          NaN              NaN       NaN         0.29  
ACN     2001        15.17        11.404394 -0.010547         0.25  
ACN     2001        15.01        11.284108 -0.000666         0.46  
ACN     2001        15.00        11.276587 -0.009333         0.27  
ACN     2001        14.8

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [11]:
# Write your code below.
import numpy as np
dd_feat=dd_engg
pandas_df = dd_feat.compute()
pandas_df = pandas_df.sort_index()
print(pandas_df.columns)
pandas_df = pandas_df.sort_values('ticker') 
pandas_df['returns_mavg10'] = pandas_df.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

print(pandas_df.head(20))

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range'],
      dtype='object')
             Date   Open   High    Low  Close  Adj Close      Volume   source  \
ticker                                                                          
ACN    2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
ACN    2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
ACN    2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
ACN    2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
ACN    2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   
ACN    2001-07-26  14.95  14.99  14.50  14.50  10.900705   6335300.0  ACN.csv   
ACN    2001-07-27  14.51  14.59  14.50  14.51  10.908223   3524000.0  ACN.csv   
ACN    2001-07-30  14.50  14.78  14.50  14.70  11.051059   3654300.0  ACN.csv   
ACN    2001-

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

> No, it was not necessary to convert the Dask DataFrame to a pandas DataFrame to calculate the moving average of returns. Converting it into pandas first, helps with the subsequent processing expecting a pandas object.
>
> Yes, it would have been better to do it in Dask, especially if we had been working with large datasets, then practicing with Dask might have been better for scalability and performance. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.